# Wood density grid search for All by random forest models in GEE

In [1]:
# import the libraries
import ee
import pandas as pd
import os
import numpy as np
import random
from random import sample
import itertools 
import geopandas as gpd
from sklearn.metrics import r2_score
from termcolor import colored # this is allocate colour and fonts type for the print title and text
from IPython.display import display, HTML

In [ ]:
#set the working directory of local drive for Grid search result table loading
# os.getcwd()
os.chdir('~/WoodDensityMapingProject')

In [3]:
# initialize the earth engine API
ee.Initialize()

In [4]:
# define the boundary geography reference
unboundedGeo = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False)

In [5]:
# define the list of predictors
propertyOfInterest = ["Aridity_Index",
                      "CHELSA_Annual_Mean_Temperature",
                      "CHELSA_Annual_Precipitation",
                      "CHELSA_Isothermality",
                      "CHELSA_Max_Temperature_of_Warmest_Month",
                      "CHELSA_Mean_Diurnal_Range",
                      "CHELSA_Mean_Temperature_of_Coldest_Quarter",
                      "CHELSA_Mean_Temperature_of_Driest_Quarter",
                      "CHELSA_Mean_Temperature_of_Warmest_Quarter",
                      "CHELSA_Mean_Temperature_of_Wettest_Quarter",
                      "CHELSA_Min_Temperature_of_Coldest_Month",
                      "CHELSA_Precipitation_Seasonality",
                      "CHELSA_Precipitation_of_Coldest_Quarter",
                      "CHELSA_Precipitation_of_Driest_Month",
                      "CHELSA_Precipitation_of_Driest_Quarter",
                      "CHELSA_Precipitation_of_Warmest_Quarter",
                      "CHELSA_Precipitation_of_Wettest_Month",
                      "CHELSA_Precipitation_of_Wettest_Quarter",
                      "CHELSA_Temperature_Annual_Range",
                      "CHELSA_Temperature_Seasonality",
                      "Depth_to_Water_Table",
                      "EarthEnvCloudCover_MODCF_interannualSD",
                      "EarthEnvCloudCover_MODCF_intraannualSD",
                      "EarthEnvCloudCover_MODCF_meanannual",
                      "EarthEnvTopoMed_Eastness",
                      "EarthEnvTopoMed_Elevation",
                      "EarthEnvTopoMed_Northness",
                      "EarthEnvTopoMed_ProfileCurvature",
                      "EarthEnvTopoMed_Roughness",
                      "EarthEnvTopoMed_Slope",
                      "EarthEnvTopoMed_TopoPositionIndex",
                      "SG_Absolute_depth_to_bedrock",
                      "WorldClim2_SolarRadiation_AnnualMean",
                      "WorldClim2_WindSpeed_AnnualMean",
                      "WorldClim2_H2OVaporPressure_AnnualMean",
                      "NDVI",
                      "EVI",
                      "Lai",
                      "Fpar",
                      "Npp",
                      "Tree_Density",
                      "PET",
                      "SG_Clay_Content_0_100cm",
                      "SG_Coarse_fragments_0_100cm",
                      "SG_Sand_Content_0_100cm",
                      "SG_Silt_Content_0_100cm",
                      "SG_Soil_pH_H2O_0_100cm",
                      "LandCoverClass_Cultivated_and_Managed_Vegetation",
                      "LandCoverClass_Urban_Builtup",
                      "Human_Disturbance",
                      "PresentTreeCover",
                      "Nitrogen",
                      "CanopyHeight",
                      "cropland",
                      "grazing",
                      "pasture",
                      "rangeland",
                      "Fire_Frequency",
                      "cnRatio",
                      "Cation",
                      "SoilMoisture",
                      "ForestAge"]
print(propertyOfInterest[0:5])

['Aridity_Index', 'CHELSA_Annual_Mean_Temperature', 'CHELSA_Annual_Precipitation', 'CHELSA_Isothermality', 'CHELSA_Max_Temperature_of_Warmest_Month']


In [6]:
# generate the classifier list based on fullParameterSpace
def classifierListsGenerator (paramterSets, randomDiscrete = True, randomNumber = 12,nTrees = 100,modelType = 'REGRESSION',bagFraction=0.632,Seed=0):
    # define an empty list to load the defined models for grid search
    classifierList = []
    if randomDiscrete:
        # check the randomNumber
        if randomNumber is None:
            print('Warning! an integer number needs to be allocated to <randomNumber>!')
        else:
            print('A randomDiscrete approach has been applied to do grid search the paramter space! \n  The random model number is: '+str(randomNumber)+' !')
            # subset the fullParameterSpace randomly with the randomNumber
            random.seed(Seed)
            randomParameterApplied = sample(paramterSets,randomNumber)
            # print(randomSubsetParameter)
            
    else:
        print('The full space of the parameter sets is being running for grid search')
        randomParameterApplied = sample(paramterSets,randomNumber)
    print('function use 20 as the default nTrees, \n You can define you own nTree value in the function argument settings!')
    # loop through the randomParameterApplied
    for ParaSet in randomParameterApplied:
        model_name = 'GridSeach_Model_'+str(ParaSet[0])+'_'+str(ParaSet[1])+'_'+str(ParaSet[2])
        # define the paramter setting of each model in the grid seach and allocate those parameters into the feature
        perRF = ee.Feature(ee.Geometry.Point([0,0])).set('ModelName',model_name,'PerClassifier',ee.Classifier.smileRandomForest(
            # the default ntrees we use 100
            numberOfTrees=nTrees,
            variablesPerSplit = ParaSet[0],
            minLeafPopulation = ParaSet[1],
            maxNodes = ParaSet[2],
            bagFraction=bagFraction).setOutputMode(modelType))
        classifierList.append(perRF)
    return(classifierList)

In [7]:
# Define the R^2 function for use with continuous valued models (i.e., regression based models)
def coefficientOfDetermination(anyVariableTable,propertyOfInterest,propertyOfInterest_Predicted):
    # Compute the mean of the property of interest
    propertyOfInterestMean = ee.Number(ee.Dictionary(ee.FeatureCollection(anyVariableTable).select([propertyOfInterest]).reduceColumns(ee.Reducer.mean(),[propertyOfInterest])).get('mean'));
    # Compute the total sum of squares
    def totalSoSFunction(f):
        return f.set('Difference_Squared',ee.Number(ee.Feature(f).get(propertyOfInterest)).subtract(propertyOfInterestMean).pow(ee.Number(2)))
    totalSumOfSquares = ee.Number(ee.Dictionary(ee.FeatureCollection(anyVariableTable).map(totalSoSFunction).select(['Difference_Squared']).reduceColumns(ee.Reducer.sum(),['Difference_Squared'])).get('sum'))
    # Compute the residual sum of squares
    def residualSoSFunction(f):
        return f.set('Residual_Squared',ee.Number(ee.Feature(f).get(propertyOfInterest)).subtract(ee.Number(ee.Feature(f).get(propertyOfInterest_Predicted))).pow(ee.Number(2)))
    residualSumOfSquares = ee.Number(ee.Dictionary(ee.FeatureCollection(anyVariableTable).map(residualSoSFunction).select(['Residual_Squared']).reduceColumns(ee.Reducer.sum(),['Residual_Squared'])).get('sum'))
    # Finalize the calculation
    r2 = ee.Number(1).subtract(residualSumOfSquares.divide(totalSumOfSquares))
    # print('I am running as well!')

    return ee.Number(r2)

In [8]:
# Define a function to take a feature with a classifier of interest
def computeCVAccuracy(featureWithClassifier,
                      propertyOfInterest,
                      modelType,
                      kFoldAssignmentFC,
                      cvFoldString,
                      classProperty,
                      accuracyMetricString,
                      extractedVariableTable):
    # Pull the classifier from the feature
    cOI = ee.Classifier(featureWithClassifier.get('PerClassifier'))
    # Create a function to map through the fold assignments and compute the overall accuracy
    # for all validation folds
    def computeAccuracyForFold(foldFeature):
        # Organize the training and validation data
        foldNumber = ee.Number(ee.Feature(foldFeature).get('Fold'))
        # print(foldNumber.getInfo())
        trainingData = extractedVariableTable.filterMetadata(cvFoldString,'not_equals',foldNumber)
        # print(trainingData.first().getInfo())
        validationData = extractedVariableTable.filterMetadata(cvFoldString,'equals',foldNumber)
        # Train the classifier and classify the validation dataset
        trainedClassifier = cOI.train(trainingData,classProperty,propertyOfInterest)
        outputtedPropName = classProperty+'_Predicted'
        classifiedValidationData = validationData.classify(trainedClassifier,outputtedPropName)
        # Create a central if/then statement that determines the type of accuracy values that are returned
        if modelType == 'CLASSIFICATION':
            # Compute the overall accuracy of the classification
            errorMatrix = classifiedValidationData.errorMatrix(classProperty,outputtedPropName,categoricalLevels)
            overallAccuracy = ee.Number(errorMatrix.accuracy())
            return foldFeature.set(accuracyMetricString,overallAccuracy)
        else:
            # Compute the R^2 of the regression
            r2ToSet = coefficientOfDetermination(classifiedValidationData,classProperty,outputtedPropName)
            return foldFeature.set(accuracyMetricString,r2ToSet)

    # Compute the accuracy values of the classifier across all folds
    accuracyFC = kFoldAssignmentFC.map(computeAccuracyForFold)
    meanAccuracy = accuracyFC.aggregate_mean(accuracyMetricString)
    tsdAccuracy = accuracyFC.aggregate_total_sd(accuracyMetricString)
    # print('I am running!')
    # Compute the feature to return
    featureToReturn = featureWithClassifier.select(['ModelName']).set('Mean_'+accuracyMetricString,meanAccuracy,'StDev_'+accuracyMetricString,tsdAccuracy)
    return featureToReturn

In [9]:
def gridSearchEarthEngine(inputTrainTable,# train data table in ee.FeatureCollection format
                          propertyOfInterest = propertyOfInterest, # list of predictors
                          classProperty = 'WdDnsty', # response varibale name in Google earth engine
                          nTrees = 20, # number of trees, default is 100
                          variablesPerSplitList = np.arange(3, 22, 3).tolist(), # list
                          minLeafPopulationList = np.arange(2, 21, 2).tolist(), # list
                          maxNodesList = np.arange(10, 101, 10).tolist(),# list
                          bagFraction = 0.632,
                          randomDiscrete = True, #boolean
                          randomNumber = 1, # if random discrete is True, you must set this value
                          foldsValue = 10,
                          modelType = 'REGRESSION',
                          cvFoldString = 'CV_Fold',
                          pyramidingPolicy = 'mean',
                          accuracyMetricString = 'R2',
                          Seeds=0):
    
    parameterLists = [variablesPerSplitList,minLeafPopulationList,maxNodesList]
    # generate the list of all the possible paramter set combinations
    fullParamterSpace = list(itertools.product(*parameterLists))
    # generate the classifer in featureColletion format
    classifierList = classifierListsGenerator(paramterSets = fullParamterSpace,
                                              randomNumber = randomNumber,
                                              nTrees = nTrees,
                                              bagFraction = 0.632,
                                              Seed=Seeds)
    
    kList = list(range(0,foldsValue))
    kFoldAssignmentFC = ee.FeatureCollection(ee.List(kList).map(lambda n: ee.Feature(ee.Geometry.Point([0,0])).set('Fold',n)))
    # print(kFoldAssignmentFC.getInfo())
    classDf = pd.DataFrame(columns = ['Mean_R2','StDev_R2','ModelName','numberOfTrees','variablesPerSplit','minLeafPopulation','bagFraction','maxNodes'])

    for rf in classifierList:
        # print(rf.getInfo())
        accuracy_feature = ee.Feature(computeCVAccuracy(rf,propertyOfInterest,modelType='REGRESSION',kFoldAssignmentFC= kFoldAssignmentFC,cvFoldString = cvFoldString,classProperty=classProperty,accuracyMetricString =accuracyMetricString,extractedVariableTable = inputTrainTable))
        # extract the parameter information
        parameterDict = rf.getInfo().get('properties',{}).get('PerClassifier').get('classifier',{})
        parameterDF = pd.DataFrame(parameterDict,index = [0])
        # extract the metrics information
        metricDict = accuracy_feature.getInfo().get('properties')
        metricDF = pd.DataFrame(metricDict,index = [0])

        # print(metricDF)
        # print(parameterDF)
        resultDF = pd.concat([metricDF, parameterDF], axis=1, sort=False)
        # print(resultDF)
        classDf = classDf.append(resultDF, sort=False)
    # sort the grid search result by descending of Mean_R2
    classDfSorted = classDf.sort_values(['Mean_R2'], ascending = False)

    # print('Top 5 grid search results:\n', classDfSorted.head(5))
    return(classDfSorted.head(1)) 

In [ ]:
# generate a ee.List to save the seeds
seedList = np.arange(167, 200, 1).tolist()
print(colored('The seeds are:', 'blue', attrs=['bold']),seedList)
print(colored('Model is running!', 'blue', attrs=['bold']))
for seed in seedList:
    inputVariableTable  = ee.FeatureCollection('users/leonidmoore/WoodDensityProject/TrainTables/Wood_Density_BufferZone_Subsampled_Train_table_Seed_'+str(seed))
    # check the information of the FeatureCollection with predictors and covariates
    print(inputVariableTable .size().getInfo())
    # print(extractedVariableTable.limit(1).getInfo())
    topModelParameter = gridSearchEarthEngine(inputTrainTable = inputVariableTable,
                                              propertyOfInterest = propertyOfInterest,
                                              classProperty = 'WdDnsty',
                                              randomNumber = 48,
                                              nTrees = 250,
                                              Seeds=seed)
    # write the top parameters table to local folder
    topModelParameter.to_csv('Data/GridSearchResultsGEE/Wood_Density_Buffer_Based_Subsample_Grid_Search_Seed_'+str(seed)+'.csv',header=True,mode='w+')
    # show the progress for the grid seach by the seed number
    print(colored('Grid search for seed:'+str(seed)+' is done!', 'blue', attrs=['bold']))


The seeds are: [167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199]
Model is running!
3443
A randomDiscrete approach has been applied to do grid search the paramter space! 
  The random model number is: 48 !
function use 20 as the default nTrees, 
 You can define you own nTree value in the function argument settings!
Grid search for seed:167 is done!
3489
A randomDiscrete approach has been applied to do grid search the paramter space! 
  The random model number is: 48 !
function use 20 as the default nTrees, 
 You can define you own nTree value in the function argument settings!
Grid search for seed:168 is done!
3501
A randomDiscrete approach has been applied to do grid search the paramter space! 
  The random model number is: 48 !
function use 20 as the default nTrees, 
 You can define you own nTree value in the function argument settings!
Grid search for seed:169 is done!


Grid search for seed:195 is done!
3490
A randomDiscrete approach has been applied to do grid search the paramter space! 
  The random model number is: 48 !
function use 20 as the default nTrees, 
 You can define you own nTree value in the function argument settings!
Grid search for seed:196 is done!
3456
A randomDiscrete approach has been applied to do grid search the paramter space! 
  The random model number is: 48 !
function use 20 as the default nTrees, 
 You can define you own nTree value in the function argument settings!
Grid search for seed:197 is done!
3479
A randomDiscrete approach has been applied to do grid search the paramter space! 
  The random model number is: 48 !
function use 20 as the default nTrees, 
 You can define you own nTree value in the function argument settings!
Grid search for seed:198 is done!
3462
A randomDiscrete approach has been applied to do grid search the paramter space! 
  The random model number is: 48 !
function use 20 as the default nTrees, 
 Yo